In [5]:
import requests
from requests.auth import HTTPBasicAuth
import hashlib
import hmac
import base64
import time
import json
import pandas as pd
import urllib3
import os
import shutil
from datetime import date, timedelta
import paramiko
import stat

# ==== SFTP connection details ====
HOSTNAME = "SGKOMSFFTAP02"
PORT     = 22
USERNAME = "svc.sftp-isrp"
PASSWORD = "5$6pL9MRn1$7{T9"

# ==== SFTP Remote Directory ====
REMOTE_DIR = "/HikVision/Incoming/"


def upload_to_sftp(local_file_path):
    """Uploads a single file to the remote SFTP path"""
    
    if not os.path.isfile(local_file_path):
        print(f"⚠ File not found, skip upload: {local_file_path}")
        return

    print(f"🔐 Connecting to SFTP...")
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(
        hostname=HOSTNAME,
        port=PORT,
        username=USERNAME,
        password=PASSWORD,
        look_for_keys=False,
        allow_agent=False
    )

    try:
        sftp = client.open_sftp()
        remote_path = os.path.join(REMOTE_DIR, os.path.basename(local_file_path))

        print(f"⬆ Uploading {local_file_path} → {remote_path}")
        sftp.put(local_file_path, remote_path)
        print(f"✅ Upload Complete: {os.path.basename(local_file_path)}")

        sftp.close()
    finally:
        client.close()




# ---------- CONFIG ----------
APP_KEY = "22793623"
APP_SECRET = "chi08BQy1ZJioFYc5tz5"
HOST = "https://118.189.49.19"

# Person ID to Code Mapping
person_id_map = {}

def create_signature(api_path):
    method = "POST"
    content_type = "application/json;charset=UTF-8"
    accept = "application/json"
    string_to_sign = f"{method}\n{accept}\n{content_type}\n{api_path}"
    return base64.b64encode(
        hmac.new(APP_SECRET.encode(), string_to_sign.encode(), hashlib.sha256).digest()
    )

def call_hik_api(api_path, payload, headers_override=None):
    url = HOST + api_path
    timestamp = str(int(time.time()))
    signature = create_signature(api_path)
    headers = {
        "Content-Type": "application/json;charset=UTF-8",
        "userId": "admin",
        "Accept": "application/json",
        "X-Ca-Key": APP_KEY,
        "X-Ca-Signature": signature,
        "X-Ca-Timestamp": timestamp
    }
    if headers_override:
        headers.update(headers_override)
    try:
        response = requests.post(url, headers=headers, json=payload, verify=False)
        return response.json()
    except Exception as e:
        print("❌ API error:", e)
        return {}

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ---------- CONFIG ----------
APP_KEY = "22793623"
APP_SECRET = "chi08BQy1ZJioFYc5tz5"
HOST = "https://118.189.49.19"

# Person ID to Code Mapping
person_id_map = {} # Initialize as a dictionary

#use 2 line below for yesterday
start = f"{(date.today() - timedelta(days=1))}T00:00:00+08:00"
end = f"{date.today()}T00:00:00+08:00"


# # use 2 line below for the day itself
# start = f"{date.today()}T00:00:00+08:00"
# end = f"{date.today()}T23:59:00+08:00"

def create_signature(api_path):
    method = "POST"
    content_type = "application/json;charset=UTF-8"
    accept = "application/json"
    string_to_sign = f"{method}\n{accept}\n{content_type}\n{api_path}"
    return base64.b64encode(
        hmac.new(APP_SECRET.encode(), string_to_sign.encode(), hashlib.sha256).digest()
    ).decode()

def call_hik_api(api_path, payload, headers_override=None):
    url = HOST + api_path
    timestamp = str(int(time.time()))
    signature = create_signature(api_path)
    headers = {
        "Content-Type": "application/json;charset=UTF-8",
        "userId": "admin",
        "Accept": "application/json",
        "X-Ca-Key": APP_KEY,
        "X-Ca-Signature": signature,
        "X-Ca-Timestamp": timestamp
    }
    if headers_override:
        headers.update(headers_override)
    try:
        response = requests.post(url, headers=headers, json=payload, verify=False)
        return response.json()
    except Exception as e:
        print("❌ API error:", e)
        return {}

def fetch_person_profiles():
    global person_id_map
    page = 1
    while True:
        payload = {"pageNo": page, "pageSize": 500, "appendInfo": [6]}
        data = call_hik_api("/artemis/api/resource/v1/person/personList", payload)
        if not isinstance(data, dict):
            print("❌ Invalid personList response (not JSON):", data)
            break
        records = data.get("data", {}).get("list", [])
        if not records:
            break
        for rec in records:
            pid = rec.get("personId")
            pcode = rec.get("personCode")
            if pid and pcode:
                person_id_map[pid] = pcode
        page += 1

def get_fr_terminal_logs(start, end, person_id=None):
    CLOCKING_DOOR_IDS = ["2996", "2997", "3727", "3263", "3468", "3470", # 4 TBY (3 IN, 1 OUT) & 2 AY LODGE
                         "3682", "3677", "3692", "3687", "3078", "3118", "3123", "3083", "3088", "3699", "3642", "3637", "3000",
                         "3704", "3010", "2987", "2985", "2992", "3632", "2998", "3732", "3709", "3015", "3024", "3652", "3647",
                         "3005", "3220", "3828", "3230", "3235", "3271", "3190", "3276", "3195", "3200", "3205", "3210", "3215",
                         "3055", "2810", "3258"] 





# def get_fr_terminal_logs(start, end, person_id=None):
#     CLOCKING_DOOR_IDS = ["83", "199", "290", "310", "315", "320", "325", "330", "335", "340",
#                             "1709", "1841", "1842", "1843", "1892", "1893",
#                             "2062", "2173", "2179", "2185", "2214", "2219", "2224", "2229", "2239",
#                             "2265", "2270", "2275", "2280", "2285", "2295", "2315", "2320", "2325",
#                             "2330", "2332", "2334", "2337", "2342", "2347", "2352", "2357",
#                             "2490", "2492", "2576", "2590", "2593", "2606",
#                             "2638", "2642", "2643", "2644", "2645", "2646",
#                             "2660", "2661", "2662", "2663", "2664", "2665", "2666", "2667", "2669", "2671",
#                             "2796", "2797", "2798", "2799", "2800", "2801", "2802", "2803", "2804", "2805", "2810",
#                             "2985", "2987", "2992", "2996", "2997", "2998", "3000", "3005", "3010", "3015", "3024", "3055",
#                             "3078", "3083", "3088", "3093", "3098", "3103", "3108", "3113", "3118", "3123",
#                             "3190", "3195", "3200", "3205", "3210", "3215", "3220", "3225", "3230", "3235",
#                             "3258", "3263", "3699", "3642", "3637", "3000", "3704", "2987",
#                             "3632", "3709", "3652", "3647", "3271", "3276", "3470" , "3468"]  # Office IDs now included


    #recently added on 11/11:

# 3699	3642	3637	3000	3704	2987	3632	3709	3652	3647	3271	3276
    logs = []
    for i in range(0, len(CLOCKING_DOOR_IDS), 10):
        batch = CLOCKING_DOOR_IDS[i:i + 10]
        page = 1
        while True:
            payload = {
                "startTime": start,
                "endTime": end,
                "eventType": 196893,
                "doorIndexCodes": batch,
                "pageNo": page,
                "pageSize": 100
            }
            if person_id:
                payload["certificateNo"] = person_id
            data = call_hik_api("/artemis/api/acs/v1/door/events", payload)
            records = data.get("data", {}).get("list", [])
            if not records:
                break
            for rec in records:
                door_id = rec.get("doorIndexCode", "")
                logs.append({
                    "Name": rec.get("personName", ""),
                    "ID": person_id_map.get(rec.get("personId", ""), rec.get("personId", "")),
                    "Door": door_id,
                    "Time": rec.get("eventTime", ""),
                    "Device": rec.get("doorName", "")
                })
            page += 1
    return logs


# --- Modified run_search function ---
def run_search(start_time, end_time, person_id=None):
    logs = []
    print(f"Searching logs for person ID: {person_id or 'All'} from {start_time} to {end_time}")

    logs.extend(get_fr_terminal_logs(start_time, end_time))

    if person_id:
        logs = [log for log in logs if log.get("ID") == person_id == person_id] # Added Plate for ANPR filtering

    logs.sort(key=lambda x: x["Time"])
    return logs # Return the logs instead of updating a global variable for display

# --- Execution ---
print("📡 Fetching person profiles...")
fetch_person_profiles()

# If you want to search for a specific person ID, pass it here
# For example: search_results = run_search(start, end, person_id="1001985")
search_results = run_search(start, end) # Calling without a specific person_id to get all logs
#search_results = run_search(start, end, person_id = "1001985")

print("\n--- Search Results ---")
if search_results:
    # Convert list of dictionaries to a Pandas DataFrame
    df = pd.DataFrame(search_results)

    # Define the output Excel file name
    output_filename = "hikvision_logs.xlsx"
    df = df.rename(columns={'Name': 'First Name', 'ID': 'Person No.','Device': 'Access Point'})
    df['Time'] = pd.to_datetime(df['Time'])
    df['Time'] = df['Time'].dt.tz_localize(None)
    # Output folder
    output_folder = r"C:\Users\Bryan.HAU\Downloads"
    # Create a "Date" column to group by day
    df['Date'] = df['Time'].dt.date
    # Loop through each unique date and save a separate Excel file
    df.drop(['Door'], axis=1, inplace = True)
    # df = df[df['Access Point'].str.contains('TBY', case=False, na=False)]
    df[df['Access Point'].str.contains('TBY', case=False, na=False) | df['Access Point'].str.contains('AY ALAUNIA LODGE iSRP', case=False, na=False)]
    # Loop through each unique date
    for day, group in df.groupby('Date'):
        # Format date as ddMMyyyy
        date_str = pd.to_datetime(day).strftime("%Y%m%d")
    
        # --- Split into IN and OUT ---
        df_in = group[group['Access Point'].str.contains("IN", case=False, na=False)]
        df_out = group[group['Access Point'].str.contains("OUT", case=False, na=False)]

        output_folder = r"C:\Users\Bryan.HAU\Downloads"
        destination_folder = r"C:\Users\Bryan.HAU\Service4 OneDrive\OneDrive - Seatrium Ltd\Digital Trace\TBY FR reports"
    
        # Save IN file if not empty
        if not df_in.empty:
            output_filename_in = f"Person Authentication IN Record_{date_str}.csv"
            output_path_in = os.path.join(output_folder, output_filename_in)
            df_in.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_in, index=False, encoding="utf-8-sig")
            print(f"✅ Saved {output_filename_in} with {len(df_in)} rows")

             # Save file in Downloads
            df_in.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_in, index=False, encoding="utf-8-sig")
            print(f"✅ Saved {output_filename_in} in {output_folder} with {len(df_in)} rows")

             # Move file to OneDrive Testing folder
            final_path_in = os.path.join(destination_folder, output_filename_in)
            shutil.move(output_path_in, final_path_in)
            print(f"📁 Moved {output_filename_in} to {final_path_in}")

    
        # Save OUT file if not empty
        if not df_out.empty:
            output_filename_out = f"Person Authentication OUT Record_{date_str}.csv"
            output_path_out = os.path.join(output_folder, output_filename_out)
            df_out.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_out, index=False, encoding="utf-8-sig")
            print(f"✅ Saved {output_filename_out} with {len(df_out)} rows")

              # Save file in Downloads
            df_out.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_out, index=False, encoding="utf-8-sig")
            print(f"✅ Saved {output_filename_out} in {output_folder} with {len(df_out)} rows")

             # Move file to OneDrive Testing folder
            final_path_out = os.path.join(destination_folder, output_filename_out)
            shutil.move(output_path_out, final_path_out)
            print(f"📁 Moved {output_filename_out} to {final_path_out}")


            df_in = group[group['Access Point'].str.contains("IN", case=False, na=False)]
            df_out = group[group['Access Point'].str.contains("OUT", case=False, na=False)]

            output_folder = r"C:\Users\Bryan.HAU\Downloads"
            destination_folder = r"C:\Users\Bryan.HAU\Service4 OneDrive\OneDrive - Seatrium Ltd\Digital Trace\TBY FR reports"

            # --------------------------
            # Process IN File
            # --------------------------
            if not df_in.empty:
                output_filename_in = f"Person Authentication IN Record_{date_str}.csv"
                output_path_in = os.path.join(output_folder, output_filename_in)
                df_in.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_in, index=False, encoding="utf-8-sig")
                print(f"✅ Saved {output_filename_in} in {output_folder} with {len(df_in)} rows")
            
                # Move to OneDrive
                final_path_in = os.path.join(destination_folder, output_filename_in)
                shutil.move(output_path_in, final_path_in)
                print(f"📁 Moved {output_filename_in} to {final_path_in}")
            
                # Upload to SFTP
                upload_to_sftp(final_path_in)


            # --------------------------
            # Process OUT File
            # --------------------------
            if not df_out.empty:
                output_filename_out = f"Person Authentication OUT Record_{date_str}.csv"
                output_path_out = os.path.join(output_folder, output_filename_out)
                df_out.drop(columns=['Date']).sort_values(by='Time').to_csv(output_path_out, index=False, encoding="utf-8-sig")
                print(f"✅ Saved {output_filename_out} in {output_folder} with {len(df_out)} rows")
            
                # Move to OneDrive
                final_path_out = os.path.join(destination_folder, output_filename_out)
                shutil.move(output_path_out, final_path_out)
                print(f"📁 Moved {output_filename_out} to {final_path_out}")
            
                # Upload to SFTP
                upload_to_sftp(final_path_out)

📡 Fetching person profiles...
Searching logs for person ID: All from 2025-12-11T00:00:00+08:00 to 2025-12-12T00:00:00+08:00

--- Search Results ---
✅ Saved Person Authentication IN Record_20251211.csv with 14556 rows
✅ Saved Person Authentication IN Record_20251211.csv in C:\Users\Bryan.HAU\Downloads with 14556 rows
📁 Moved Person Authentication IN Record_20251211.csv to C:\Users\Bryan.HAU\Service4 OneDrive\OneDrive - Seatrium Ltd\Digital Trace\TBY FR reports\Person Authentication IN Record_20251211.csv
✅ Saved Person Authentication OUT Record_20251211.csv with 11571 rows
✅ Saved Person Authentication OUT Record_20251211.csv in C:\Users\Bryan.HAU\Downloads with 11571 rows
📁 Moved Person Authentication OUT Record_20251211.csv to C:\Users\Bryan.HAU\Service4 OneDrive\OneDrive - Seatrium Ltd\Digital Trace\TBY FR reports\Person Authentication OUT Record_20251211.csv
✅ Saved Person Authentication IN Record_20251211.csv in C:\Users\Bryan.HAU\Downloads with 14556 rows
📁 Moved Person Authentica